# Holder Wallet Conformance Flows (v3) - CTWalletCrossDeferred

# 0.0 Initial setup

## 0.1 Setup conformance

In [1]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [2]:
from ebsi_wallet.did_key import KeyDid, PublicKeyJWK
import uuid

# generate crypto seed
crypto_seed = b'helloworld'

key_did = KeyDid(seed=crypto_seed)

# generate keypair
key_did.create_keypair()

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    y=key_did.public_key_jwk['y']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

Decentralised identifier:  did:key:z2dmzD81cgPx8Vki7JbuuMmFYrWPgYoytykUZ3eyqht1j9KbptQwoaj2VP1S6Ahzo7REFCT4NBTPYdQinCZbCcyoqWKi9Q2uEW36DNSXhCwiYnGz6BAZkzytQAEBE5cPidCGnadH4SsLDbSZeG2SEChrqvQpdK4Mk8H32vs3B5g8Wr7kcc


## 1.1 Initiate Credential Issuance

In [3]:
from ebsi_wallet.util import parse_query_string_parameters_from_url
from ebsi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

credential_type =  CredentialTypes.CTWalletCrossDeferred
# qr_code_data = await fetch_credential_offer(key_did._did, credential_type)
qr_code_data = "openid-credential-offer://?credential_offer_uri=https%3A%2F%2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fissuer-mock%2Foffers%2Ff008d867-c9ad-4e3d-b96b-8b6f28e0f048"
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_server)
console.log("Authorization server configuration: ", auth_server_configuration)


[16:09:20] Credential offer URI:                                                                   ]8;id=758605;file:///tmp/ipykernel_27003/3787712226.py\3787712226.py]8;;\:]8;id=631363;file:///tmp/ipykernel_27003/3787712226.py#16\16]8;;\
           https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/offers/f008d867-c9ad-4e3d-b9                 
           6b-8b6f28e0f048                                                                                         

{'credential_issuer': 'https://api-conformance.ebsi.eu/conformance/v3/issuer-mock', 'credentials': [{'format': 'jwt_vc', 'types': ['VerifiableCredential', 'VerifiableAttestation', 'CTWalletCrossDeferred'], 'trust_framework': {'name': 'ebsi', 'type': 'Accreditation', 'uri': 'TIR link towards accreditation'}}], 'grants': {'authorization_code': {'issuer_state': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW95dHlrVVozZXlxaHQxajlLYnB0UXdvYWoyVlAxUzZBaHpvN1JFRkNUNE5CVFBZZFFpbkNaYkNjeW9xV0tpOVEydUVXMzZETlNYaEN3aVluR3o2QkFaa3p5dFFBRUJFNWNQaWRDR25hZEg0U3NMRGJTWmVHMlNFQ2hycXZRcGRLNE1rOEgzMnZzM0I1ZzhXcjdrY2MiLCJjcmVkZW50aWFsX3R5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc0RlZmVycmVkIl0sImlhdCI6MTY4NzI1NzUxMCwiZXhwIjoxNjg3MjU3ODEwLCJpc3MiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIiwiYXV

[16:09:21] Credential offer:                                                                       ]8;id=396353;file:///tmp/ipykernel_27003/3787712226.py\3787712226.py]8;;\:]8;id=359992;file:///tmp/ipykernel_27003/3787712226.py#19\19]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               credentials=[                                                                                       
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'CTWalletCrossDeferred'                                                                 
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       }                                                                                           
                   }                                                                                               
               ],                                                                                                  
               grants={                                                                                            
                   'authorization_code': {                                                                         
                       'issuer_state':                                                                             
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNF                 
           ZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1                 
           BnWW95dHlrVVozZXlxaHQxajlLYnB0UXdvYWoyVlAxUzZBaHpvN1JFRkNUNE5CVFBZZFFpbkNaYkNjeW9xV0tpO                 
           VEydUVXMzZETlNYaEN3aVluR3o2QkFaa3p5dFFBRUJFNWNQaWRDR25hZEg0U3NMRGJTWmVHMlNFQ2hycXZRcGRL                 
           NE1rOEgzMnZzM0I1ZzhXcjdrY2MiLCJjcmVkZW50aWFsX3R5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiw                 
           iVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc0RlZmVycmVkIl0sImlhdCI6MTY4NzI1NzUxMC                 
           wiZXhwIjoxNjg3MjU3ODEwLCJpc3MiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hb                 
           mNlL3YzL2lzc3Vlci1tb2NrIiwiYXVkIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5ldS9jb25mb3Jt                 
           YW5jZS92My9hdXRoLW1vY2siLCJzdWIiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW9                 
           5dHlrVVozZXlxaHQxajlLYnB0UXdvYWoyVlAxUzZBaHpvN1JFRkNUNE5CVFBZZFFpbkNaYkNjeW9xV0tpOVEydU                 
           VXMzZETlNYaEN3aVluR3o2QkFaa3p5dFFBRUJFNWNQaWRDR25hZEg0U3NMRGJTWmVHMlNFQ2hycXZRcGRLNE1rO                 
           EgzMnZzM0I1ZzhXcjdrY2MifQ.FAuRkpx30DiTAnVaGpt3sEuVLxHbQFdWUCYULrMkXjqGFLajGEVPLgoJF62Ay                 
           uDxkJIUp_SEgZfqLs-941WfKg'    

[16:09:22] Credential issuer configuration:                                                        ]8;id=651683;file:///tmp/ipykernel_27003/3787712226.py\3787712226.py]8;;\:]8;id=982673;file:///tmp/ipykernel_27003/3787712226.py#22\22]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               authorization_server='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                    
               credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/cre                 
           dential',                                                                                               
               deferred_credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer                 
           -mock/credential_deferred',                                                                             
               credentials_supported=[                                                                             
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAuthorisationToOnboard'                                                      
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       },                                                                                          
                       'display': [                                                                                
                           {'name': 'Verifiable Authorisation to onboard', 'locale': 'en-GB'}                      
                       ]                                                                                           
                   },                                                                                              
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAccreditation',                                                              
                           'VerifiableAccreditationToAttest'                                                       
                       ],                                                                                          
                       'trust_framework':

           Authorization server configuration:                                                     ]8;id=593896;file:///tmp/ipykernel_27003/3787712226.py\3787712226.py]8;;\:]8;id=354755;file:///tmp/ipykernel_27003/3787712226.py#25\25]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=[                                                                                     
                   'https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                          
               ],                                                                                                  
               issuer='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                                  
               authorization_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/au                 
           thorize',                                                                                               
               token_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/token',                    
               jwks_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/jwks',                           
               scopes_supported=['openid'],                                                                        
               response_types_supported=['vp_token', 'id_token'],                                                  
               response_modes_supported=['query'],                                                                 
               grant_types_supported=['authorization_code'],                                                       
               subject_types_supported=['public'],                                                                 
               id_token_signing_alg_values_supported=['ES256'],                                                    
               request_object_signing_alg_values_supported=['ES256'],                                              
               request_parameter_supported=True,                                                                   
               request_uri_parameter_supported=True,                                                               
               token_endpoint_auth_methods_supported=['private_key_jwt'],                                          
               request_authentication_methods_supported={                                                          
                   'authorization_endpoint': ['request_object']                                                    
               },                                                                                                  
               vp_formats_supported={                                                                              
                   'jwt_vp': {'alg_values_supported': ['ES256']},                                                  
                   'jwt_vc': {'alg_values_supported': ['ES256']}                                                   
               },                                                                                                  
               subject_syntax_types_supported=['did:key', 'did:ebsi'],                                             
               subject_syntax_types_discriminations=['did:key:jwk_jcs-pub', 'did:ebsi:v1'],                        
               subject_trust_frameworks_supported=['ebsi'],                                                        
               id_token_types_supported=['subject_signed_id_token', 'attester_signed_id_token']                    
           )                                                                                                       

## 1.2 Perform authorization request and obtain ID token request

In [4]:
from ebsi_wallet.siop_auth.util import (
    perform_authorization, 
    AuthorizationRequestQueryParams,
    get_authorization_response_query_params,
    generate_code_challenge,
    generate_code_verifier
)
import uuid
import json

state = str(uuid.uuid4())
nonce = str(uuid.uuid4())
code_verifier = generate_code_verifier()

authorization_details = [{"type":"openid_credential","format":"jwt_vc","types":credential_offer.credentials[0].get("types"),"locations": ["https://api-conformance.ebsi.eu/conformance/v3/issuer-mock"]}]
redirect_uri = "http://localhost:8080"
client_metadata = {"vp_formats_supported":{"jwt_vp":{"alg":["ES256"]},"jwt_vc":{"alg":["ES256"]}},"response_types_supported":["vp_token","id_token"],"authorization_endpoint":redirect_uri}
authorization_request_query_params = AuthorizationRequestQueryParams(
    response_type="code",
    scope="openid",
    state=state,
    client_id=key_did._did,
    authorization_details=json.dumps(authorization_details, separators=(',', ':')),
    redirect_uri=redirect_uri,
    nonce=nonce,
    code_challenge=generate_code_challenge(code_verifier),
    code_challenge_method="S256",
    client_metadata=json.dumps(client_metadata, separators=(',', ':')),
    issuer_state=credential_offer.grants.get("authorization_code").get("issuer_state")
)

auth_resp = await perform_authorization(auth_server_configuration.authorization_endpoint, authorization_request_query_params)
auth_resp_uri = str(auth_resp).split("Location': '")[1].split("'")[0]
console.log("Authorization response URI: ", auth_resp_uri)
auth_resp_query_params = get_authorization_response_query_params(auth_resp_uri)
console.log("Authorization response query params: ", auth_resp_query_params)

[16:09:27] Authorization response URI:                                                             ]8;id=896094;file:///tmp/ipykernel_27003/1836098038.py\1836098038.py]8;;\:]8;id=919492;file:///tmp/ipykernel_27003/1836098038.py#34\34]8;;\
           http://localhost:8080?state=c1c2a065-7323-4d92-8183-a80e1159fb4c&client_id=https%3A%2F%                 
           2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock&redirect_uri=https%3A%2F%2Fapi                 
           -conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock%2Fdirect_post&response_type=id_toke                 
           n&response_mode=direct_post&scope=openid&nonce=0338f0d1-91fb-466e-811f-78d428c52ee1&req                 
           uest_uri=https%3A%2F%2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock%2Frequest                 
           _uri%2F67c98ca3-245d-4647-9d5e-7ebf0ce6f22f                                                             

           Authorization response query params:                                                    ]8;id=820179;file:///tmp/ipykernel_27003/1836098038.py\1836098038.py]8;;\:]8;id=610891;file:///tmp/ipykernel_27003/1836098038.py#36\36]8;;\
           AuthorizationResponseQueryParams(                                                                       
               state='c1c2a065-7323-4d92-8183-a80e1159fb4c',                                                       
               client_id='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                               
               redirect_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                 
           ,                                                                                                       
               response_type='id_token',                                                                           
               response_mode='direct_post',                                                                        
               scope='openid',                                                                                     
               nonce='0338f0d1-91fb-466e-811f-78d428c52ee1',                                                       
               request_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/request_uri/6                 
           7c98ca3-245d-4647-9d5e-7ebf0ce6f22f',                                                                   
               presentation_definition='',                                                                         
               request=''                                                                                          
           )                                                                                                       

## 1.3 Send ID token

In [5]:
from ebsi_wallet.siop_auth.util import send_id_token_response

id_token = key_did.generate_id_token(auth_server_uri=auth_resp_query_params.client_id, nonce=auth_resp_query_params.nonce)
auth_code_response = await send_id_token_response(auth_resp_query_params.redirect_uri, id_token, auth_resp_query_params.state)
auth_code_response = str(auth_code_response).split("Location': '")[1].split("'")[0]
state = parse_query_string_parameters_from_url(auth_code_response).get("state")[0]
auth_code = parse_query_string_parameters_from_url(auth_code_response).get("code")[0]
console.log("Authorization code: ", auth_code)

[16:09:36] Authorization code:  6981e4dc-fc04-4590-b97c-914eb3945dd8                                ]8;id=555895;file:///tmp/ipykernel_27003/1546471797.py\1546471797.py]8;;\:]8;id=118058;file:///tmp/ipykernel_27003/1546471797.py#8\8]8;;\

## 1.4 Exchange code for access token

In [6]:
from ebsi_wallet.siop_auth.util import exchange_auth_code_for_access_token

token_uri = auth_resp_query_params.client_id + "/token"
access_token_response = await exchange_auth_code_for_access_token(token_uri, key_did._did, auth_code, code_verifier)
console.log("Access token response: ", access_token_response)

[16:09:54] Access token response:                                                                    ]8;id=141477;file:///tmp/ipykernel_27003/474231906.py\474231906.py]8;;\:]8;id=661049;file:///tmp/ipykernel_27003/474231906.py#5\5]8;;\
           AccessTokenResponse(                                                                                    
               access_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWV               
           ZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6ImJiYWRhNmM1LWY1NGMtNDg2MC1hYWUyLThhMWY0N               
           GU4NjEyYSIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudG               
           lhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV               
           1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVy               
           aWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc0RlZmVycmVkIl19XSwiY19ub25jZSI6ImEyZWI1MDAxL               
           WM5MzItNDAxOS04YTJmLTA0MDRhYjNjMzcxNSIsImNfbm9uY2VfZXhwaXJlc19pbiI6ODY0MDAsImNsaWVudF9pZC               
           I6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJ               
           WUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFF               
           QkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyJ9LCJpc3MiOiJod               
           HRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2F1dGgtbW9jayIsImF1ZCI6WyJodH               
           RwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInN1YiI6ImR               
           pZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWUDFT               
           NkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y               
           1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsImlhdCI6MTY4NzI1Nz               
           U5NCwiZXhwIjoxNjg3MzQzOTk0fQ.PhtdmIbnUzr0uP_w8oSZ-NhyQQmEQGsFfpfY3h0_aP6u3gHhf9hBws4S4F6_               
           cILUbLWX9-xvUYhSJ_qYdZG7hw',                                                                            
               token_type='Bearer',                                                                                
               expires_in=86400,                                                                                   
               id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUWl               
           GbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6ImEyYTA5NTEyLWYzNzEtNDRmNi1hZGRlLTQ0OWUxN2Q2N               
           jU0YSIsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2               
           NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2J               
           wdFF3b2FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJB               
           Wmt6eXRRQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwiY               
           XVkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2               
           FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJBWmt6eXR               
           RQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwiaWF0Ijox               
           Njg3MjU3NTk0LCJleHAiOjE2ODcyNTc3MTR9.Tvekc62ho2TWuAkz9Cvgp_qPmvp26RwlYLxFCsdeQxq3cVG_ZawL               
           tKM17fSStfSQeZIHhdRziaUlhMQZhCgiSg',                                                                    
               c_nonce='a2eb5001-c932-4019-8a2f-0404ab3c3715',                                                     
               c_nonce_expires_in=86400     

## 1.5 Request credential to get acceptance token to initiate deferred flow 

In [7]:
from ebsi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_request(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
acceptance_token_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_offer.credentials[0].get("types"))
console.log("Acceptance token response: ", acceptance_token_response)## 1.6 Request credential using deferred endpoint

[16:10:10] Credential request JWT:                                                                   ]8;id=358038;file:///tmp/ipykernel_27003/380321948.py\380321948.py]8;;\:]8;id=215627;file:///tmp/ipykernel_27003/380321948.py#4\4]8;;\
           eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVW               
           jNleXFodDFqOUticHRRd29hajJWUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1               
           hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjV               
           nOFdyN2tjYyN6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2FqMlZQ               
           MVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJBWmt6eXRRQUVCR               
           TVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwidHlwIjoib3Blbm               
           lkNHZjaS1wcm9vZitqd3QifQ.eyJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1h               
           bmNlL3YzL2lzc3Vlci1tb2NrIiwiZXhwIjoxNjg3MzQ0MDEwLCJpYXQiOjE2ODcyNTc2MTAsImlzcyI6ImRpZDprZ               
           Xk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWUDFTNkFoem               
           83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZEN               
           HbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsIm5vbmNlIjoiYTJlYjUwMDEt               
           YzkzMi00MDE5LThhMmYtMDQwNGFiM2MzNzE1In0.TMZ0_WkUUx42sQXnkWejbUM5CfLkw8KaqHiePiSFrYiRzP3R0               
           cayxumTNMM_lx4D1aKN6xLCscpNoXew5LwhHw                                                                   

[16:10:11] Acceptance token response:                                                                ]8;id=847348;file:///tmp/ipykernel_27003/380321948.py\380321948.py]8;;\:]8;id=853606;file:///tmp/ipykernel_27003/380321948.py#6\6]8;;\
           {'acceptance_token': 'gQm4WYEdjZtQx-zxwlLSSIGN-Oq-tZ8rex24gToN6i4'}                                     

## 1.6 Request credential using deferred endpoint

In [8]:
from ebsi_wallet.siop_auth.util import send_deferred_credential_request
from time import sleep

console.log("Waiting for 5 seconds...")
sleep(5)
console.log("Sending deferred credential request...")
credential_response = await send_deferred_credential_request(credential_issuer_configuration.deferred_credential_endpoint, acceptance_token_response.get("acceptance_token"))
print("Credential response: ", credential_response)

[16:10:16] Waiting for 5 seconds...                                                                 ]8;id=330252;file:///tmp/ipykernel_27003/1677177840.py\1677177840.py]8;;\:]8;id=615725;file:///tmp/ipykernel_27003/1677177840.py#4\4]8;;\

[16:10:21] Sending deferred credential request...                                                   ]8;id=151371;file:///tmp/ipykernel_27003/1677177840.py\1677177840.py]8;;\:]8;id=280052;file:///tmp/ipykernel_27003/1677177840.py#6\6]8;;\

Credential response:  {'format': 'jwt_vc', 'credential': 'eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkI3VPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJqdGkiOiJ2YzplYnNpOmNvbmZvcm1hbmNlI2Q3ZmQ0ODM0LWZiYWQtNGFhNy05YWRkLWQyNTVhYThhNWQ0YyIsInN1YiI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsImlzcyI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkIiwibmJmIjoxNjg3MjU3NjExLCJleHAiOjE2ODczNDQwMTEsImlhdCI6MTY4NzI1NzYxMSwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiXSwiaWQiOiJ2YzplYnNpOmNvbmZvcm1hbmNlI2Q3ZmQ0ODM0LWZiYWQtNGFhNy05YWRkLWQyNTVhYThhNWQ0YyIsInR5cGUiOlsiVmVyaWZpYWJsZUNyZWRlbnRpYWwiLCJWZXJpZmlhYmxlQXR0ZXN0YXRpb24iLCJDVFdhbGxldENyb3NzRGVmZXJyZWQiXSwiaXNzdWVyIjoiZGlkOmVic2k6emhKQVJqUE42OWNFdGdQeEh